In [14]:
# Connect robot
import qi
from naoqi import ALProxy
from ProductDetails import *

import time
import numpy as np
import matplotlib.pyplot as plt
import cv2

session = qi.Session()
robot_ip = '192.168.1.31:9559'
virtual_robot = 'localhost:37677'

session.connect('tcp://' + virtual_robot)

In [15]:
# Use API services

tts = session.service("ALTextToSpeech")
nav = session.service("ALNavigation")
atts = session.service("ALAnimatedSpeech")

video_proxy = session.service("ALVideoDevice")
dialog = session.service("ALDialog")
memory = session.service("ALMemory")
tablet = session.service("ALTabletService") 

RuntimeError: Cannot find service 'ALTabletService' in index

In [16]:
# Moving

#motion = session.service("ALMotion")
#motion = ALProxy("ALMotion", 'localhost', 34353)
#moving = motion.post.moveTo(2.0, 0.0, 0.0)
#tts.say("Lets explore!")
#motion.wait(moving, 0)
#tts.say("I have finished exploring!")

tts.say("hi")


def listen_for(topic):
    result = []
    
    def callback(value):
        if value and len(value) > 0:
            result.append(value)
    
    # Try to unload any topic that already exists
    topics = dialog.getAllLoadedTopics()
    for t in topics:
        try:
            dialog.unloadTopic(t)
        except: 
            pass # do nothing if there was an error

    # Load the new topic
    topic_name = dialog.loadTopicContent(topic)
    
    # Activate the topic and subscribe to user speech
    dialog.activateTopic(topic_name)
    dialog.subscribe('newtopic')
    subscriber = memory.subscriber('Dialog/LastInput')
    subscriber.signal.connect(callback)
    
    # Keep waiting until some results have arrived
    while len(result) == 0:
        time.sleep(0.1)
    
    # Clean up by unloading the topic
    dialog.unsubscribe('newtopic')
    dialog.deactivateTopic(topic_name)
    dialog.unloadTopic(topic_name)
    
    
    # Return the first result that was collected
    return result[0]



In [ ]:
# Begin interaction
#u:(~greetings) %greet ^start(animations/Stand/Gestures/Hey_1) Hello human! ^wait(animations/Stand/Gestures/Hey_1) ~begin
productsAvailable = ['chocolate', 'beverage', 'fruit']
interact_topic = """

topic: ~interact()
language: enu

u: (_[chocolate beverage fruit]) Cool! Let me see if we have $1 in the store. brb $item=$1 ^stayInScope
u: (e:Dialog/NotUnderstood) sorry, we don't have that item in stock.  ^stayInScope
"""

tts.say("Hello, what are you looking for today?")

while True:
    result = listen_for(interact_topic)

    print 'The robot heard', result

    if result in productsAvailable:
        print('available')    

In [ ]:
def get_frame(proxy, camera_idx=0, resolution_idx=1, colorspace_idx=11, fps=20):
    if not proxy.isCameraOpen(camera_idx):
        proxy.openCamera(camera_idx)

    if not proxy.isCameraStarted(camera_idx):
        video_proxy.startCamera(camera_idx)
    
    if resolution_idx in [3, 4]:
        # max fps for these resolutions
        fps = 1
    
    sub = proxy.subscribeCamera(
        "get_frame_sub",
        camera_idx,
        resolution_idx,
        colorspace_idx,
        fps
    )
    
    np_image = None
    
    try:
        timeout = 3
        start_time = time.time()
        result = None
        while time.time() - start_time < timeout and result is None:
            result = proxy.getImageRemote(sub)

        if result:
            buffer_image = result[6]
            
            if result[2] == 3:
                img_shape = (result[1], result[0], result[2])
                im_format = np.uint8
            else:
                img_shape = (result[1], result[0])
                im_format = np.uint16
            
            temp = np.frombuffer(buffer_image, im_format)
            np_image = np.reshape(temp, img_shape)
    except Exception as e:
        print(e)
    finally:
        proxy.unsubscribe(sub)
    
    return np_image
    

In [ ]:
# current image from robot camera
current_frame = get_frame(video_proxy)
plt.imshow(current_frame)
plt.show()